<a href="https://colab.research.google.com/github/ambideXtrous9/Seq2Seq-Attention-QA/blob/main/Seq2Seq_with_AttentionQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet transformers
!pip install --quiet pytorch-lightning
!pip install --quiet tokenizers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch
torch.set_float32_matmul_precision('high')
import random
import numpy as np
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer as Tokenizer
from torch.optim import AdamW

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
MODEL_NAME = 'google/flan-t5-small'

In [5]:
tokenizer = Tokenizer.from_pretrained(MODEL_NAME)

In [6]:
path = '/content/drive/MyDrive/MTP CODE/NewsQA_SPAN.feather'

In [7]:
df = pd.read_feather(path)

In [8]:
df = df.iloc[:5000]

In [9]:
train_df, val_df = train_test_split(df,test_size=0.2)
val_df, test_df = train_test_split(val_df,test_size=0.5)

In [10]:
class QADataset(Dataset):
  def __init__(self,data : pd.DataFrame,tokenizer : Tokenizer,source_max_token_len : int = 200,target_max_token_len : int = 15):

    self.tokenizer = tokenizer
    self.data = data
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def __len__(self):
    return len(self.data)
  
  def __getitem__(self,index : int):
    data_row = self.data.iloc[index]

    question_context_encodings = tokenizer(
        data_row['question'],
        data_row['paragraph'],
        max_length = self.source_max_token_len,
        padding = "max_length",
        truncation = "only_second",
        add_special_tokens = True,
        return_tensors = "pt")
    
    answer_encodings = tokenizer(
        data_row['answer'],
        max_length = self.target_max_token_len,
        padding = "max_length",
        truncation = True,
        add_special_tokens = True,
        return_tensors = "pt")
    
    
    return {'input_ids': question_context_encodings['input_ids'].flatten(),
            'labels': answer_encodings['input_ids'].flatten()}


In [11]:
sample_dataset = QADataset(df,tokenizer)

In [12]:
tokenizer.vocab_size

32100

In [13]:
for data in sample_dataset:
  print(type(data['input_ids']))
  print(data['input_ids'])
  break

<class 'torch.Tensor'>
tensor([ 2645,    19,     8,  5037,  2090,    13,  8951,    49,   397,    15,
         5826,    58,     1,    96, 13898,   127,     7,    54,   169,     3,
            9,  2711,    13,   789, 13237,    11,   731,  8225, 14410,  2814,
         2731,    12,   918,     3,     9, 15812,    21,    70,  4833,   976,
          845,  1813,   157,  2375, 10729,   138,     6,  5037,  2090,     6,
         8951,    49,   397,    15,  5826,     5,  5421, 13015,     7,   243,
         4367,   228,  2153,   710,    18,  1987,     6,  2768,    18,  1987,
           11,   307,    18,  1987,   789, 13237,    28,   128,  8543,  3069,
          494,     5,     3, 17229,     6,     3,     9,   386,    18,  7393,
        20792,  3259,   133,   428,  4367,     3,     9,  6339,    13,  1877,
         2712,     6,     3,     9,    80,    18,  1201,   332,  2876,   133,
         6339,     3, 19708,  4704,    11,     3,     9,  9445,  1201,   789,
         1034,  6339,     7,     3, 27865

In [14]:
class QADataModule(pl.LightningDataModule):
  def __init__(self,train_df , val_df, test_df,tokenizer : Tokenizer,batch_size : int = 8,source_max_token_len : int = 200,target_max_token_len : int = 15):
    super().__init__()
    self.batch_size = batch_size
    self.train_df = train_df
    self.test_df = test_df
    self.val_df = val_df
    self.tokenizer = tokenizer
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def setup(self,stage=None):
    self.train_dataset = QADataset(self.train_df,self.tokenizer,self.source_max_token_len,self.target_max_token_len)
    self.val_dataset = QADataset(self.val_df,self.tokenizer,self.source_max_token_len,self.target_max_token_len)
    self.test_dataset = QADataset(self.test_df,self.tokenizer,self.source_max_token_len,self.target_max_token_len)
    

  def train_dataloader(self):
    return DataLoader(self.train_dataset,batch_size = self.batch_size,shuffle=True,num_workers=2)

  def val_dataloader(self):
    return DataLoader(self.val_dataset,batch_size = self.batch_size,num_workers=2)

  def test_dataloader(self):
    return DataLoader(self.test_dataset,batch_size = self.batch_size,num_workers=2)   

In [71]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, bidirectional=True)

        self.fc_hidden = nn.Linear(hidden_size * 2, hidden_size)
        self.fc_cell = nn.Linear(hidden_size * 2, hidden_size)
        self.dropout = nn.Dropout(p)

    def forward(self, x):
        # x: (seq_length, N) where N is batch size

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)

        encoder_states, (hidden, cell) = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)

        # Use forward, backward cells and hidden through a linear layer
        # so that it can be input to the decoder which is not bidirectional
        # Also using index slicing ([idx:idx+1]) to keep the dimension
        hidden = self.fc_hidden(torch.cat((hidden[0:1], hidden[1:2]), dim=2))
        cell = self.fc_cell(torch.cat((cell[0:1], cell[1:2]), dim=2))

        return encoder_states, hidden, cell


In [72]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, p):
        super(Decoder, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(hidden_size * 2 + embedding_size, hidden_size, num_layers)

        self.energy = nn.Linear(hidden_size * 3, 1)
        self.fc = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(p)
        self.softmax = nn.Softmax(dim=0)
        self.relu = nn.ReLU()

    def forward(self, x, encoder_states, hidden, cell):

        #print("x = ",x.shape)
        x = x.unsqueeze(0)
        # x: (1, N) where N is the batch size
        #print("x uns = ",x.shape)
        embedding = self.dropout(self.embedding(x))
        #print("x embed = ",embedding.shape)

        sequence_length = encoder_states.shape[0]
        h_reshaped = hidden.repeat(sequence_length, 1, 1)
        # h_reshaped: (seq_length, N, hidden_size*2)

        energy = self.relu(self.energy(torch.cat((h_reshaped, encoder_states), dim=2)))
        # energy: (seq_length, N, 1)

        attention = self.softmax(energy)
        # attention: (seq_length, N, 1)

        # attention: (seq_length, N, 1), snk
        # encoder_states: (seq_length, N, hidden_size*2), snl
        # we want context_vector: (1, N, hidden_size*2), i.e knl
        #print("attention = ",attention.shape)
        #print("encoder_states = ",encoder_states.shape)
        context_vector = torch.einsum("snk,snl->knl", attention, encoder_states)

        #print(context_vector.shape)
        #print(embedding.shape)

        rnn_input = torch.cat((context_vector, embedding), dim=2)
        # rnn_input: (1, N, hidden_size*2 + embedding_size)

        outputs, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        # outputs shape: (1, N, hidden_size)

        predictions = self.fc(outputs).squeeze(0)
        # predictions: (N, hidden_size)

        return predictions, hidden, cell


In [73]:
class Seq2Seq(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

        self.criterion = nn.CrossEntropyLoss(ignore_index=0)


    def forward(self, source, target, teacher_force_ratio=0.5):
        
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = tokenizer.vocab_size

        #print("source = ",source.shape)
        #print("target = ",target.shape)

        outputs = torch.zeros(target_len, batch_size, target_vocab_size)
        encoder_states, hidden, cell = self.encoder(source)

        # First input will be <SOS> token
        x = target[0]

        #print("x_target = ",x.shape)

        for t in range(1, target_len):
            # At every time step use encoder_states and update hidden, cell
            output, hidden, cell = self.decoder(x, encoder_states, hidden, cell)

            # Store prediction for current time step
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)

            # With probability of teacher_force_ratio we take the actual next word
            # otherwise we take the word that the Decoder predicted it to be.
            # Teacher Forcing is used so that the model gets used to seeing
            # similar inputs at training and testing time, if teacher forcing is 1
            # then inputs at test time might be completely different than what the
            # network is used to. This was a long comment.
            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs

    def training_step(self,batch,batch_idx):
        inp_data = batch['input_ids'].transpose(0, 1)
        target = batch['labels'].transpose(0, 1)

        #print("TS = ",target.shape)
        # Forward prop
        output = model(inp_data, target)

        # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
        # doesn't take input in that form. For example if we have MNIST we want to have
        # output to be: (N, 10) and targets just (N). Here we can view it in a similar
        # way that we have output_words * batch_size that we want to send in into
        # our cost function, so we need to do some reshapin. While we're at it
        # Let's also remove the start token while we're at it
        output = output[1:].reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        target = target.to(output.device)


        loss = self.criterion(output, target)

        
        self.log("train_loss",loss,prog_bar=True,logger=True)
        return loss

    def validation_step(self,batch,batch_idx):
        inp_data = batch['input_ids'].transpose(0, 1)
        target = batch['labels'].transpose(0, 1)
        output = model(inp_data, target)
        output = output[1:].reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)
        target = target.to(output.device)
        loss = self.criterion(output, target)
        self.log("val_loss",loss,prog_bar=True,logger=True)
        return loss

    def configure_optimizers(self):
        return AdamW(self.parameters(),lr = 0.0001)


In [74]:
checkpoint_callback = ModelCheckpoint(
    dirpath = 'checkpoints',
    filename = 'best_cp',
    save_top_k = 1,
    verbose = True,
    monitor = 'val_loss',
    mode = 'min'
)

In [75]:
BATCH_SIZE = 64
N_EPOCHS = 20

data_module = QADataModule(train_df,val_df,test_df,tokenizer,batch_size = BATCH_SIZE)
data_module.setup()

In [76]:
encoder = Encoder(input_size=tokenizer.vocab_size,
                  embedding_size=300,
                  hidden_size=256,
                  num_layers=1,
                  p=0.0)

In [77]:
decoder = Decoder(input_size=tokenizer.vocab_size,
                  embedding_size=300,
                  hidden_size=256,
                  num_layers=1,
                  p=0.0,
                  output_size=tokenizer.vocab_size)

In [78]:
model = Seq2Seq(encoder,decoder)

In [79]:
trainer = pl.Trainer(devices=-1, accelerator="gpu",
    callbacks=[checkpoint_callback],
    max_epochs = N_EPOCHS
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [80]:
trainer.fit(model,data_module)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /content/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type             | Params
-----------------------------------------------
0 | encoder   | Encoder          | 11.0 M
1 | decoder   | Decoder          | 19.0 M
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
30.0 M    Trainable params
0         Non-trainable params
30.0 M    Total params
120.046   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 63: 'val_loss' reached 6.37638 (best 6.37638), saving model to '/content/checkpoints/best_cp-v1.ckpt' as top 1


...

INFO:pytorch_lightning.utilities.rank_zero:Epoch 17, global step 1134: 'val_loss' reached 4.60409 (best 4.60409), saving model to '/content/checkpoints/best_cp-v1.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 18, global step 1197: 'val_loss' reached 4.56490 (best 4.56490), saving model to '/content/checkpoints/best_cp-v1.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 19, global step 1260: 'val_loss' reached 4.53400 (best 4.53400), saving model to '/content/checkpoints/best_cp-v1.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [81]:
import torch

In [82]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [83]:
cppath = '/content/checkpoints/best_cp.ckpt'

In [84]:
trained_model = Seq2Seq.load_from_checkpoint(cppath,encoder=encoder,decoder=decoder)
trained_model.freeze()

In [85]:
def predict(question):
    trained_model.eval()

    # Tokenize the source text
    source_tokens = tokenizer(
        question['question'],
        question['paragraph'],
        max_length = 200,
        padding = "max_length",
        truncation = "only_second",
        add_special_tokens = True,
        return_tensors = "pt")['input_ids'].flatten().to(device)

    sentence_tensor = source_tokens.unsqueeze(1).to(device)

    trained_model.to(device)

    # Encode the source text
    with torch.no_grad():
        encoder_states, hidden, cell = trained_model.encoder(sentence_tensor)


    max_length = 15
    # Initialize the predicted sentence
    outputs = [0]

    # Generate the output sequence token by token
    for _ in range(max_length):

        previous_word = torch.LongTensor([outputs[-1]]).to(device)
        
        # Decode the next token
        with torch.no_grad():
            output, hidden, cell = trained_model.decoder(previous_word, encoder_states, hidden, cell)
            best_guess = output.argmax(1).item()
            

        # Add the predicted token to the predicted sentence
        outputs.append(best_guess)

    
    print(outputs)

    # Convert the predicted sentence back to text
    predicted_text = tokenizer.decode(outputs,skip_special_tokens=True,clean_up_tokenization_spaces=True)

    return predicted_text

In [86]:
sample_question = test_df.iloc[1]

In [87]:
sample_question['question']


'What board did Mallya resign from in 2017?'

In [88]:
sample_question['answer']


'UBL'

In [89]:
predict(sample_question)

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


''